In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pytz
import urllib as ur
import json

## Requisitando Estacionamentos, carros e coordenadas

In [ ]:
response = ur.request.urlopen('https://bookit.modo.coop/api/v2/car_list').read().decode('UTF-8')
json_cars = json.loads(response)

response = ur.request.urlopen('https://bookit.modo.coop/api/v2/location_list').read().decode('UTF-8')
json_location = json.loads(response)

In [ ]:
car_coord = pd.DataFrame(list(json_cars['Response']['Cars'].keys()), columns=['car_id'], dtype='int')
car_coord['location'] = [0] * len(car_coord)
car_coord['lat'] = [0] * len(car_coord)
car_coord['lon'] = [0] * len(car_coord)

for i in range(len(car_coord)):
    car_coord['location'].iloc[i] = int(json_cars['Response']['Cars'][str(car_coord['car_id'].iloc[i])]['Location'][0]['LocationID'])
    car_coord['lat'].iloc[i] = float(json_location['Response']['Locations'][str(car_coord['location'].iloc[i])]['Latitude'])
    car_coord['lon'].iloc[i] = float(json_location['Response']['Locations'][str(car_coord['location'].iloc[i])]['Longitude'])

## Fazendo a média da quantidade de viagens para cada hora

In [ ]:
# CSV criado a partir dos dados coletados do arquivo ModoApi_Data_Filter
dfTravels = pd.read_csv('travels_v2.csv')
dfTravels = dfTravels.sort_values(by='car_id')

In [ ]:
def cont_travels(df):
    """
    Função para calcular a quantidade de viagens por hora de cada carro
    
    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame com dados das viagens registradas
        
    Returns
    ---------
    df_count : pandas.DataFrame
        DataFrame com os dados organizados com horas como colunas e linhas sendo id dos veículos.
        Os seus valores representam a quantidade de viagens efetuadas em dada hora por tal veículo.
    
    """
    df_cont = []
    id_prox = df['car_id'].iloc[1]
    id_atual = df['car_id'].iloc[0]
    tempo = []

    for i in range(len(df)):
        try:
            id_prox = df['car_id'].iloc[i+1]
            id_atual = df['car_id'].iloc[i]
        except:
            pass

        # Se irá mudar de id então é registrado o somatório dos intervalos de tempo estacionado
        # i == len(df)-1 para não pular o ultimo
        if (id_prox != id_atual or i == len(df)-1):
            
            hour = datetime.datetime.fromtimestamp(df['start'].iloc[i]).hour
            
            auxHour = [0] * 24
            auxHour[hour] += 1
            tempo.append(auxHour)
            
            #Somando todas a quantidade de ocorrências de cada horário
            tempo = pd.DataFrame(tempo)
            
            tempo = list(pd.Series.sum(tempo))
            
            tempo = [id_atual] + tempo
            
            df_cont.append(tempo)
            tempo = []
        else:
            # Verificando a hora de inicio da viagem e somando em uma lista que representa as 24h
            # Evita mais de uma viagen por hora
            hour = datetime.datetime.fromtimestamp(df['start'].iloc[i]).hour
            hour_anterior = datetime.datetime.fromtimestamp(df['start'].iloc[i-1]).hour
            
            if (hour == hour_anterior):
                continue
            
            auxHour = [0] * 24
            auxHour[hour] += 1
            #Armazenando a quantidade de viagens em dada hora
            tempo.append(auxHour)
    
    #Labels das colunas
    labels = list(range(-1,24))
    [format(x,'02d') for x in labels]
    labels[0] = 'car_id'
    
    df_cont = pd.DataFrame(df_cont, columns=labels)
    df_cont = df_cont.sort_values(by=['car_id'])
    
    return df_cont

In [ ]:
hours = cont_travels(dfTravels)

In [ ]:
# Preparando o dataframe para colocar a location, latitude ,longitude e numero de carros
hours['lat'] = [0]*len(hours)
hours['lon'] = [0]*len(hours)
hours['location'] = [0]*len(hours)
hours['n_cars'] = [1]*len(hours)

In [ ]:
# Colocando as coordenadas de cada carro
for i in range(len(hours)):
    try:
        coord = car_coord[car_coord['car_id'] == hours['car_id'].iloc[i]]
        hours['lat'].iloc[i] = coord['lat'].iloc[0]
        hours['lon'].iloc[i] = coord['lon'].iloc[0]
        hours['location'].iloc[i] = coord['location'].iloc[0]
    except Exception as e:
        # Carros que sairam da frota
        print(e)
        print('id:'+str(hours['car_id'].iloc[i]))

hours = hours.sort_values(by='location')

In [ ]:
# Somando todos os valores para cada estação
# A cada loop verifica se ainda existe repetições de id
while(True in list(hours.duplicated(subset=['location']))):
    i = 0
    while i < len(hours):
        try:
            if (hours['location'].iloc[i] == hours['location'].iloc[i+1]):
                print('Antes:')
                print(len(hours))
                
                # Percorre todas as 24 somando a quantidade de viagens de cada carro da estação
                for j in range(24):
                    hours[j].iloc[i] = hours[j].iloc[i] + hours[j].iloc[i+1]
                
                # Adicionando mais um ao numero de carros da estação
                hours['n_cars'].iloc[i] = hours['n_cars'].iloc[i] + 1 
                
                # Retirando a linha já analisada
                hours = hours.drop(hours.index[i+1])
                hours.index = range(len(hours))
                
                print('Depois:')
                print(len(hours))
        except Exception as e:
            print(e)
            break
        i+=1

In [ ]:
# Ordenando por estacionamento
hours = hours.sort_values(by='location')

# Dividindo pela quantidade de dias e numero de carros
for i in range(len(hours)):
    for j in range(24):
        hours[j].iloc[i] = hours[j].iloc[i] / 31

## Normalizando os dados

In [ ]:
data = pd.DataFrame()
aux  = pd.DataFrame()

# Organizando todos os dados em uma lista
for i in range(0,24):
    hora = [i] * len(hours)
    hora = pd.DataFrame(hora)
    
    hours.dropna(how='any', axis=0, inplace=True)
    
    aux = pd.concat([hours['car_id'], hours[i], hora], axis=1)
    aux.columns = ['car_id', 'mean', 'hour']
    
    data = data.append(aux)

# Normalizando pelo maior
maior = data['mean'].max()
data['mean'] = data['mean'] / maior

In [ ]:
# Reorganizando o dataframe com os dados já normalizados
for i in range(24):
    aux = data[data['hour'] == i]
    hours[i] = aux['mean']

In [ ]:
# Ordenando e concatenando os valores com as coordenadas
hours = hours.sort_values(by='car_id')
car_coord = car_coord.sort_values(by='car_id')

aux_csv = hours
aux_csv.dropna(how='any', axis=0, inplace=True)

In [ ]:
# Csv's de cada hora
for i in range(24):
    aux_csv[['lat', 'lon', i]].to_csv('ViagensPorHora/Hour'+str(i)+'_v2.csv')

## Plotagem em mapas de calor de cada hora

In [ ]:
import geoplotlib as gpl
from geoplotlib.utils import read_csv, BoundingBox, DataAccessObject

In [ ]:
# Imprimindo todas as 24 horas
for i in range(0,24):
    
    hora = str(i)
    
    # Lendo csv com dados de tempo estacionado médio, latitude, longitude de cada estacionamento
    location = pd.read_csv('ViagensPorHora/Hour'+hora+'_v2.csv', usecols=[1,2,3])
    data = location
    # Multiplicando os valores por um escalar para se tornarem mais visíveis
    location[hora] = location[hora] * 100
    location_aux = []
    
    
    # Utilizando um auxiliar para gerar repetições de incidencias para a plotagem no mapa de calor
    for i in range(len(location)):
        for j in range(int(location[hora].iloc[i])):
            location_aux.append([location['lat'].iloc[i], location['lon'].iloc[i]])
    
    location_aux = pd.DataFrame(location_aux, columns=['lat', 'lon'])
    
    # Vancouver    
    
    gpl.kde(location_aux, bw=3, cut_below=1e-4, cmap='jet', alpha=150 )
#     data = DataAccessObject(pd.DataFrame({'lat': [],'lon': []}))
#     gpl.hist(data, scalemin=0, scalemax=100, cmap='jet', colorscale='lin', alpha=190)
    
    # Coordenadas para o mapa focar em Vancouver
    lat = pd.DataFrame([49.246292, 49.262428, 49.24966])
    lon = pd.DataFrame([-123.11554, -123.116226, -123.04464])
    
    gpl.set_bbox(BoundingBox.from_points(lon[0], lat[0]))
    gpl.request_zoom(12)
    gpl.set_window_size(1280,700)
    gpl.savefig('ViagensPorHora/ViagensPorHoraPNGs/ViagensTodos/vanc_'+hora+'_v2')
    
    # Victoria
    
    gpl.kde(location_aux, bw=3, cut_below=1e-4, cmap='jet', alpha=150 )
#     data = DataAccessObject(pd.DataFrame({'lat': [],'lon': []}))
#     gpl.hist(data, scalemin=0, scalemax=100, cmap='jet', colorscale='lin', alpha=190)
    
    # Coordenadas para o mapa focar em Victoria
    lat = pd.DataFrame([48.42666, 48.44344, 48.44560])
    lon = pd.DataFrame([-123.36027,-123.35853,-123.33673])
    
    gpl.set_bbox(BoundingBox.from_points(lon[0], lat[0]))
    gpl.request_zoom(13)
    gpl.set_window_size(1280,700)
    
    gpl.savefig('ViagensPorHora/ViagensPorHoraPNGs/ViagensTodos/vic_'+hora+'_v2')